<a href="https://colab.research.google.com/github/probabll/ntmi-tutorials/blob/main/NBC-theory-quizzes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Guide

Check the guide carefully before starting.

## ILOs

After completing this lab you should be able to 

* state facts about NBC's factorisation, parameterisation, parameter estimation, and predictive queries

**General notes**

* In this notebook you are expected to use $\LaTeX$. 




## Table of contents


* [Text Classifier](#textcls)
* [Generative Classifier](#gencls)
    * [Naive Bayes Classifier](#NBC)
    * [Classification](#classification)
    * [Parameter Estimation](#MLE)
* [Complexity](#complexity) 



## How to use this notebook

This can work as a theory recap, but also as a way to assess your knowledge. There are many quizzes with solutions, these are not at the level of exam questions, but this knowledge is certainly part of what's necessary to answer exam questions. 

# <a name="textcls"> Text Classifiers

In NLP we often want to design systems that can read a piece of text and categorise it as an instance of one of a finite set of classes. We call those systems *text classifiers* (or text categorisers).
    
It is seldom the case that a piece of text can be reasonably mapped to *a single category*, for various reasons that we have already discussed: language is ambiguous, we often lack context (about the situation, the writer, the reader), and many implicit attributes of text are subjective by nature (e.g., one's opinion about a product). So, in practice we face text classification as a pipeline: we combine a *statistical model* and a *decision rule*.
    
The statistical model realises the mapping from a given piece of text to a *conditional probability distribution* (cpd) over the categories available. The decision rule uses the information in this distribution to elect a single category that is shown to the user.
    
---

Consider the case of *sentiment classification* with three sentiment levels (i.e., negative, neutral, and positive). 

A **probabilistic sentiment classifier** maps a document $x \in \mathcal X$ to a probability distribution over the sample space $\mathcal Y = \{-1, 0, +1\}$, where -1 stands for negative, 0 stands for neutral, and +1 stands for positive sentiment. Note that we call $x$ a document, but it could be a sentence, a paragraph, or any granularity we like. A document $x$ is a sequence of words $x=\langle w_{1}, \ldots, w_{l} \rangle$, or $w_{1:l}$ for short, where $l = |x|$ is the sequence length. Each word belongs to a vocabulary $\mathcal W$ of known words, and the vocabulary size is $V$. 
The space of all possible documents $\mathcal X$ is the space of all sequences made of words from $\mathcal W$, and these sequences can be of any size, a set like that is denoted $\mathcal W^*$.

So, formally, we have a random variable $X$ taking on documents in the set $\mathcal X$, a random variable $Y$ taking on classes in the set $\mathcal Y$, and a random variable $W$ taking on words in a vocabulary $\mathcal W$. A statistical model then establishes a map from any one $x \in \mathcal X$ to the cpd $P_{Y|X=x}$.

Why does it make sense to see a classifier as a probability distribution? Well, there are a few technical reasons (we are generally very good at learning probability distributions, whereas other types of functions may be more difficult to learn), but here is an intuitive reason: probability distributions capture uncertainty due to the data (e.g., noise), due to the task definition (e.g., the task is subjective), or due to our inability to model the true underlying cognitive processes behind language understanding. See the example below.

We may say `no way` 
* in response to news about a global pandemic, in which case, we probably mean to express a negative sentiment about it;
* in response to our friend's startup being listed amongst the top-10 startups in the Netherlands, in which case we probably mean to express a positive sentiment about it;
* in response to a classmate asking `will you skip next NTMI's class?`, and in this case `no way` might just be an objective `no`, without a negative or positive sentiment.

If all we are given is the text `no way` and no additional context, nor any information about the speaker or the situation, then we cannot really determine the sentiment (not even as humans). This illustrates ambiguity/uncertainty that is inherent to the task/data. The best we can do is to pack a few assumptions into a model, use some data to estimate this model, and then let this model quantify the probability with which `no way` might express `negative`, `neutral`, or `positive` sentiment. 

As we said in the course, the probability of a certain sentiment $y \in \{-1, 0, 1\}$ given a piece of text $x$ is not a real attribute of $x$ in the world. Rather, is is a quantification of the uncertainty that a *model of sentiment analysis* can make given text and given all of the design assumptions we made. 


So, let's **design** one such model. We will pick sentiment analysis as the example, but other text classification problems fit in the same framework as long as the target classes come from a finite set of disjoint categories (e.g., sentiments, or topics, or types of named-entities, or spam vs not spam, or product categorisation in online shops, etc). Technically such sets are called *countably finite sets*.


**Goal** We want a conditional distribution $P_{Y|X=x}$ that, given some piece of text $x$ (e.g., a sentence, a paragraph, or a document), quantifies our model's beliefs in each of the classes in the sample space $\mathcal Y$.

**Challenge** Think about it for a moment, we need a distribution over a finite number of classes, let's say we have $K$ classes. This should not be too difficult right? We have already learnt about the Categorical distribution, so we could start by attempting to use something like 

\begin{align}
    Y | X=x &\sim \mathrm{Cat}(\boldsymbol\theta^{(x)}) \qquad \text{with }\boldsymbol\theta^{(x)} \in \Delta_{K-1}
\end{align}

That is, for each $x$, we have a $K$-dimensional probability vector $\boldsymbol\theta^{(x)}$ where $P_{Y|X}(y|x) = \theta^{(x)}_y$ is the probability, under this model, that $x$ should be labelled with $y$.
Recall that we called this a *tabular* representation of the Categorical cpd, the name comes from the fact that the collection of parameters necessary to prescribe all the Categorical distributions in this definition can be store in something that looks like a table (see the last section of T1).

**Exercises with solution** What is the problem with attempting to use a tabular representation of the conditional distributions $P_{Y|X}$?

<details>
    <summary><b>SOLUTION</b></summary>

The space of all documents $\mathcal X$ is practically infinite, if every conditioning document $x$ requires its own cpd with its own independent parameter vector, we will need infinitely many parameter vectors. This could never be stored. Even if we knew a clever data structures that can store such object efficiently, we could never estimate infinitely many vectors reliably, since we will only ever observe finitely many data points.
    
We might tell ourselves, well let's use *only* the documents that we have seen during training. In that case, we would never be able to classify a new document, and analysing new documents is the whole point of text classification in NLP.
    
---
    
</details>



There are at least two different approaches to this problem. The *generative* approach, which we discuss in *this* notebook, and the *discriminative* approach, which you will encounter in T3.


# <a name="gencls"> Generative Classifiers

A generative classifier is built upon a *joint distribution* $P_{YX}$ over the joint sample space $\mathcal Y \times \mathcal X$. We can use this model to *infer* a distribution $P_{Y|X=x}$ over target classes for any given $X=x$, and we can do so *on demand* (that is, whenever we observe a certain document $x$) without ever storing (and thus never having to estimate) one probability value for each and every possible pair $(x, y) \in \mathcal X \times \mathcal Y$.

**Exercise with solution** Suppose $\mathcal X$ is the space of all documents (all of them, no matter the length). Suppose $\mathcal Y = \{-1, 0, 1\}$. Explain in words what the joint sample space $\mathcal X \times \mathcal Y$ consists of and give 2 examples of outcomes in it. How large this space is?







<details>
    <summary><b>SOLUTION</b></summary>
    
The joint sample space is made of the cross product of the two sets. As the set of all documents is infinite, the joint sample space is also infinite. The elements in it are pairs of the kind $(x, y)$ where $x\in \mathcal X$ is a document, and $y \in \mathcal Y$ is one of $3$ categories. Here are some examples of outcomes in it
    
* (`i love dogs`, +1)
* (`i love dogs`, 0)
* (`i love dogs`, -1) 
    
Note that all outcomes are in there, whether or not they are plausible, even something as unreasonable as (`cats are better than dogs`, +1) is in there, as well as ill-formed sentences (`i am i am i`, +1).
    
---
    
</details>



It should not be obvious that using a joint distribution $P_{YX}$ rather than a collections of cpds $P_{Y|X}$ will make the problem any simpler. In fact, if we were to store one probability value per assignment $(X=x, Y=y)$, we would end up with the exact same problem as before (the tables would be much too large, we would be able to store them, and we would be able to estimate them effectively). 

But, a joint distribution, as it turns out, gives us an opportunity: to make simplifying assumptions about how the probability of an assignment *decomposes* in terms of probabilities of outcomes simpler than whole documents. The technical term here is **factorisation**, we will *factorise* the probability of $(X=x, Y=y)$ using cpds involving simpler variables (or rather, variables whose sample spaces are *finite* and hopefully much smaller).

<details>
    <summary><b>Factorising</b></summary>
    Factorise means to decompose using a product. You've done this before. For example, we can factorise the expression $ab + ac$ into $a\times (b +c)$.
    
You have also factorised in terms of pre-specified factors, for example, factorise 12 using prime numbers: $12 = 2\times2\times3$.
    
This is more or less what we will be doing: we will define some elementary *factors* and express probability values using those simpler factors. For us, the factors themselves are probability values, but they concern events from a smaller event space. The rules that allow us to recombine the simpler factor in order to answer whatever probability query are the rules of probability (i.e., chain rule, conditional probability, and marginal probability).
    
---
    
</details>

To specify a joint distribution that's tractable to represent and estimate, we start with the **chain rule**. 

**Joint distribution (first attempt)** 

\begin{align}
    P_{YX}(y, x) &= P_Y(y)P_{X|Y}(x|y)
\end{align}

This first attempt factors the probability of $y$ first, and then the probability of generating $x$ conditioned on $y$ being its label. This may seem counterintuitive at first, but imagine given that $y=-1$ you may indeed generate text that's very particular to negative sentiment, and not all too similar to what you would generate had $y$ been $+1$. Of course, there are exceptions to this, like our illustration with `no way`, but the intuition is still fairly reasonable. 

This factorisation does not *really* nail the problem just yet, but not that it does achieve something interesting. Whereas there are infinitely many cpds of the kind $P_{Y|X=x}$, one for each possible value of $x \in \mathcal X$, there are only $K$ cpds of the kind $P_{X|Y=y}$, one for each possible value of $y \in \mathcal Y$. If we can figure out a way to deal with the fact that the sample space of $P_{X|Y=y}$ is infinite (that is, we can generate any one of an inifinite set of possible documents), we should be able to get to a feasible design.

We achieved something here: we only need to specify $|\mathcal Y|$ cpds. But we are still stuck with the fact that a tabular representation for each one of the $K$ cpds of $P_{X|Y=y}$ would remain a very large object.

This time the *complex* outcome is on the generating side of the probability distribution (not on the conditioning side), so let's see if chain rule can help us once more:

\begin{align}
    P_{X|Y}(w_{1:l}|y) &= \prod_{i=1}^l P_{W|YH}(w_i|y, w_{<i})
\end{align}

where the random variable $W$ denotes a random word and the random variable $H$ denotes the history of random words from left-to-right (by assumption we go from left-to-right, but any order is valid under chain rule). The notation $w_{<i}$ denotes the sequence of words before the $i$th word.

This application of chain rule does a good thing and a bad thing. First, it simplifies the generating side (each factor now is responsible for assigning probability to 1 word in context), which is good for the sample space $\mathcal W$ is finite (it has $V$ words in it). But, it complicates the conditioning side again by pushing the history into it, and the problem with the history is that it's potentially very long (and it grows longer as we advance $i$ along the document). So now again we have to estimate a very large number of cpds: one for each possible value of the conditioning context $(y, w_{<i})$ in $P_{W|YH}$.

It looks like we cannot avoid having to deal with a very large space of options. And indeed, *without* making simplifying assumptions we cannot. 

But, these two applications of chain rule did achieve something remarkable, they show us that we can express the probability value of a labelled document $P_{YX}(y, x)$ using a product of other probability values, each probability value for a random variable with finite sample space.

\begin{align}
    P_{YX}(y, w_{1:l}) &= P_Y(y) \times \prod_{i=1}^l P_{W|YH}(w_i|y, w_{<i})
\end{align}

**Exercise with solution** What simplifying assumption could we make about how words depend on one another in a document in order to simplify the conditional factors $P_{W|YH}$?

<details>
    <summary><b>SOLUTION</b></summary>
    
We used chain rule to break the complex piece of text into smaller pieces, now we can make conditional independence assumptions to make these smaller pieces themselves simpler cpds. 
    
For example, let's make the conditional independence assumption that any two words are independent of one another given the class.
    
Mathematically this is expressed as $W_i \perp W_j \mid Y=y$ for $i \neq j$ in $X=w_{1:l}$. 
    
And the factorisation becomes:    
    
\begin{align}
    P_{YX}(y, w_{1:l}) &\overset{\text{ind.}}{=} P_Y(y) \prod_{i=1}^l \underbrace{P_{W|Y}(w_i|y)}_{\text{simpler}}
\end{align}
    
*Remark I.* If you have a hard time understanding this bit, have a look at the prep-work for HC2a, you might have missed some important background there.
    
*Remark II.* We use $\overset{\text{ind.}}{=}$ when an equality only holds under some independence assumption. This is different from chain rule, for example, which always holds.  
    
---
    
</details>



So far so good, but the original problem was to assign a distribution over classes given a document. Perhaps you can already see how the joint distribution above will help us get to the conditional we wanted all along. If not, don't worry, read on :)

## <a name="NBC"> Naive Bayes Classifier
    
The naive Bayes classifier (NBC) is called that way because it makes a strong conditional independence assumption (that's the 'naive' part of the name) and because it uses Bayes rule to infer the conditional probability $P_{Y|X}(y|x)$ from the joint distribution $P_{YX}$.

    
NBC factorises the **joint probability** of $(y, x)$ as:
    
\begin{align}
    P_{YX}(y, w_{1:l}) &\overset{\text{ind.}}{=} P_Y(y) \prod_{i=1}^l P_{W|Y}(w_i|y)
\end{align}

The simpler factor $P_Y$ is modelled as a Categorical distribution over classes.
The other factors, $P_{W|Y=y}$ are each a Categorical cpd devined over $V$ possible words, so $P_{W|Y}$ is a collection of $K$ Categorical cpds.


Here is the complete generative story as well as the parameterisation in terms of Categorical cpds:
    
1. Sample a target class from a prior distribution: $Y \sim \mathrm{Cat}(\phi_1, \ldots, \phi_K)$
2. Condition on $y$, and sample a word for each position $i$ from the class-conditioned distribution $W|Y=y \sim \mathrm{Cat}(\boldsymbol\pi^{(y)})$ with $\boldsymbol\pi^{(y)} \in \Delta_{V-1}$ until you draw a stop symbol (i.e., end-of-sequence or EOS).


Concretely, this is the probability value that the model assigns to a pair $(y, x)$:
    
\begin{align}
    P_{YX}(y, x) &\overset{\text{NBC}}{=} \mathrm{Cat}(y|\phi_{1:C}) \prod_{i=1}^l \mathrm{Cat}(w_i|\pi^{(y)}_{1:v}) \\
    &= \phi_y \prod_{i=1}^l \pi^{(y)}_{w_i}
\end{align}

<details>
    <summary><b>Note on meaning of generative story</b></summary>
    
If the idea of a "generative story" confuses you, don't worry you are not alone. Let's try and clarify that. The story tells how we *assume* the observations came about, it does not tell us how the observations did come about.
    
We did not actually drew samples from this distribution and ended up with a dataset filled with perfectly valid labelled documents, rather, real people labelled documents and we just downloaded a digital version of their hardwork.
    
But, now that we have a story that could have generated the data, in the sense that the data are in the support of the joint probability distribution we describe, we can assess the probability with which we would have generated the observed data. This quantity has practical uses, for example, in maximum likelihood estimation we use this quantity to try and find the parameters $\boldsymbol\phi$ and $\boldsymbol\pi$ that maximise the probability of this model ever generating the observed data.
    
</details>    


<details>
    <summary><b>What's an end-of-sequence symbol?</b></summary>
    
As we shall see in the course, models based on generative stories often need a criterion to "stop" their generation process. This is relevant after training, when we sometimes use the model to generate data; and it is also relevant for training, in that this criterion may help the model learn something useful (for example, the length of a document or something like that). 
    
The end-of-sequence symbol is a special symbol that's added to the vocabulary and used to mark the end of a sentence. Common choices are `</s>` and `-EOS-` for these sequences are unlikely to belong to the vocabulary of any language, but you can choose anything you like (as long as it is not a word in your dataset).

Before we estimate the parameters of our model, we make sure to add this EOS symbol to the end of our sequences. In the future we may need other symbols, such as BOS (begin-of-sequence), etc.
    
</details>    

**Exercise with solution** Express the probability that the NBC assigns to (`no way`, +1) in terms of the parameters of the model? How about (`no , no way`, -1)?

<details>
    <summary><b>SOLUTION</b></summary>

*Advice.* Read the question carefully. Oftentimes people start trying to compute the MLE solution, and they get confused because we gave them no data. The question is not about parameter estimation. Rather, the question asks you to express the probability of those data points in terms of the parameters of the model (the numerical values of those parameters and where they come from is not important for this). 
    
First, we pad the document with EOS symbol, i.e., we treat the document as `no way EOS`, then we plug into the formula for NBC getting 
    
\begin{equation}
    \phi_{+1} \times \pi^{(+1)}_{\text{no}} \times \pi^{(+1)}_{\text{way}} \times \pi^{(+1)}_{\text{EOS}}
\end{equation}
    
It's similar for the other case,     
\begin{equation}
    \phi_{-1} \times \left(\pi^{(-1)}_{\text{no}} \right)^2 \times \pi^{(-1)}_{\text{,}} \times \pi^{(-1)}_{\text{way}} \times \pi^{(-1)}_{\text{EOS}}
\end{equation}    
but note that we use a different cpd, this time conditioning on -1 rather than +1, we also have the probability of the comma (given -1), and the probability of `no` (given -1) factors in twice.     
    
---
    
</details>


**Exercise with solution** Can we use the NBC to assign probability to a document $x$ for which we do not know the class? If so, explain how and compute the probability of `no way`, make sure to express it in terms of the parameters of the model. 

<details>
    <summary><b>SOLUTION</b></summary>
    
The NBC is a joint distribution $P_{YX}$, if we are given only one outcome $X=x$, and not the other, we can always use *marginalisation* to compute $P_{X}(x)$:
    
\begin{align}
    P_X(x) &= \sum_{y \in \mathcal Y} P_{YX}(y, x) \\
    &=  \sum_{y \in \mathcal Y} P_{Y}(y) \prod_{i=1}^l P_{W|Y}(w_i|y) \\
    &=  \sum_{y \in \mathcal Y} \phi_{y} \prod_{i=1}^l \pi^{(y)}_{w_i}
\end{align}
    
Then the probability of `no way` is
\begin{align}
    P_X(\langle \text{no}, \text{way}, \text{EOS} \rangle) 
    &= \phi_{-1} \times \pi^{(-1)}_{\text{no}} \times \pi^{(-1)}_{\text{way}} \times \pi^{(-1)}_{\text{EOS}} \\
    &+ \phi_{0} \times \pi^{(0)}_{\text{no}} \times \pi^{(0)}_{\text{way}} \times \pi^{(0)}_{\text{EOS}} \\
    &+\phi_{+1} \times \pi^{(+1)}_{\text{no}} \times \pi^{(+1)}_{\text{way}} \times \pi^{(+1)}_{\text{EOS}}
\end{align}
    

**Warning** If this exercise was challenging, check out the prep-work for HC2a, in particular, the introduction to probabilistic graphical models.
   
---
    
</details>



**Exercise with solution** How do we obtain a cpd $P_{Y|X=x}$ given some input text? Give the expression in terms of parameters of the model.

<details>
    <summary><b>SOLUTION</b></summary>
    
Via *Bayes rule*!
    
\begin{align}    
    P_{Y|X}(y|x) &= \frac{P_{YX}(y, x)}{P_{X}(x)} \\
    &= \frac{P_Y(y)\prod_{i=1}^l P_{W|Y}(w_i|y)}{P_{X}(x)} \\
\end{align}    

Note that we can compute all terms in this formula: $P_Y(y)$ is $\phi_y$, $P_{W|Y}(w_i|y)$ is $\pi^{(y)}_{w_i}$, and we've just solved $P_X(x)$ in the previous exercise.
   
</details>

---

## <a name="classification"> Classification

How do we actually classify using the NBC? That's a fair question, and if you solved the previous quiz, you sort of already found the answer.

The algorithm that maps from a probability distribution $P_{Y|X=x}$ to a single outcome is called a **decision rule**, literally because it is a made up rule that makes decisions for us. The most popular decision rule in text classification is the *most probable class* rule, which classifies a document $x$ with the label that receives highest probability under the model. 

Essentially, when we are given a new document $x_*$, we can solve the following search problem:


\begin{align}
y^* &= \mathrm{argmax}_{k\in \mathcal Y} ~ P_{Y|X}(k|x_*)
\end{align}

That is, we are looking for the class $k$ in $\mathcal Y$ that receives maximum *posterior probability* $P_{Y|X}(k|x_*)$, this is also called the *mode* of the distribution.
    
In fact, we can solve an even simpler problem if we apply the definition of conditional probabilities:
\begin{align}
y^* &= \mathrm{argmax}_{k\in \mathcal Y} ~ \frac{P_{YX}(k, x_*)}{P_X(x_*)}
\end{align}
as we are searching for the value of $k$ while keeping the document fixed, the denominator does not change the result and can be ignored:
\begin{align}
y^* &= \mathrm{argmax}_{k\in \mathcal Y} ~ P_{YX}(k, x_*)
\end{align}
    
This says, the most probable class a-posteriori (that is, after observing the document) is the class for which the joint probability $P_{YX}(k, x_*)$ is highest.

We can then substitute in our model definition for the joint probability    
\begin{align}
y^* &= \mathrm{argmax}_{k\in \mathcal Y} ~ P_{YX}(k, x_*) \\
    &= \mathrm{argmax}_{k\in \mathcal Y} ~ P_{Y}(k) \prod_{i=1}^{l_*} P_{W|Y}(w_i|k) \\
\end{align}    
and finally, substitute in our model parameters:
\begin{align}    
y^*    &= \mathrm{argmax}_{k\in \mathcal Y} ~  \phi_k \prod_{i=1}^{l_*} \pi_{w_i}^{(k)} \\
\end{align}
   

**Exercise with solution** If $\phi_{-1}=\phi_{+1} = \frac{2}{5}$, $\phi_0 = \frac{1}{5}$ and some of the parameters $\pi^{(y)}$ are given in the rows of the following table

| $Y$ | all | the | same | EOS | ... |
|-----|-----|-----|------|-----|-----|
| -1  | a   | c   | b    | d   | ... |
| 0   | a   | c   | 2.5b   | d   | ... |
| 1   | a   | c   | b    | d   | ... |

What is the most probable label of the document `no freaking way EOS`?

<details>
    <summary><b>Solution</b></summary>

Let's start by expressing the joint probabilities $P_{YX}(y, \text{all the same EOS})$ for each possible value of $y$:
* $y=-1$: $\frac{2}{5} \times a \times c\times b \times d = \frac{2}{5}abcd$
* $y=0$: $\frac{1}{5} \times a \times c \times 2.5b  \times d = \frac{2.5}{5}abcd$
* $y=+1$: $\frac{2}{5} \times a \times c \times b \times d = \frac{2}{5}abcd$

We should now pick the label that is assigned maximum probability when paired with this document, that would be $y=0$.

See that even though we had a prior preference in favour of labels other than neutral, given the evidence in the document, the posterior prefers the neutral label.
    
</details>    

## <a name="MLE"> Parameter Estimation

We know how NBC factorises, i.e., 

\begin{equation}
P_{YX}(y, w_{1:l}) = P_Y(y)\prod_{i=1}^l P_{W|Y}(w_i|y)
\end{equation}

we have chosen a parameterisation of our cpds, i.e., 

\begin{align}
P_{YX}(y, w_{1:l}) &= \mathrm{Cat}(y|\phi_{1:K})\prod_{i=1}^l \mathrm{Cat}(w_i|\pi^{(y)}_{1:V}) \\
&= \phi_{y} \prod_{i=1}^l \pi^{(y)}_{w_i}
\end{align}

now we need only estimate the numerical values of the parameters $\boldsymbol \phi$ and $\boldsymbol \pi$.

If we are given a dataset of $N$ labelled observations $\mathcal D = \{ (x^{(n)}, y^{(1)}), \ldots, (x^{(N)}, y^{(N)}) \}$, where each document $x^{(n)}$ has size $l_n$, we can do so via maximum likelihood estimation.

**Exercise with solution** What is the MLE for $\phi_{y}$ for some $y$ in $\mathcal Y$?

<details>
    <summary><b>SOLUTION</b></summary>
    
    
\begin{align}
\phi_y = \frac{\mathrm{count}_{Y}(y)}{\sum_{k \in \mathcal Y}\mathrm{count}_Y(k)} = \frac{\mathrm{count}_{Y}(y)}{N}
\end{align}
    
where we use $\mathcal D$ to determine the counts as shown below
    
\begin{equation}
    \mathrm{count}_Y(k) = \sum_{n=1}^N [y^{(n)} = k]
\end{equation}

For a *balanced* dataset, that is, a dataset with equal number of labelled instances for each class, $Y \sim \mathrm{Cat}(\phi_{1:K})$ becomes the uniform distribution, that is, $\phi_y = 1/K$ for all $y \in \mathcal Y$.

---    
    
</details>



**Quiz** What is the MLE for $\pi^{(y)}_w$ for some $y$ in $\mathcal Y$ and $w$ in $\mathcal W$?

<details>
    <summary><b>SOLUTION</b></summary>
    
    
\begin{align}
\pi^{(y)}_{w} = \frac{\mathrm{count}_{YX}(y, w)}{\sum_{v \in \mathcal W}\mathrm{count}_{YX}(y, v)}
\end{align}
    
where we use $\mathcal D$ to determine the counts as shown below
    
\begin{equation}
    \mathrm{count}_{YX}(k, v) = \sum_{n=1}^N \sum_{i=1}^{l_n} [y^{(n)} = k] \times [w^{(n)}_{i} = v]
\end{equation}
    
---
    
</details>



For the class-conditioned distributions over the vocabulary, in particular, we normally use a *smoothing technique* to reserve some probability mass to unseen words (words outside the training data $\mathcal D$). This will prevent the estimated model from assigning zero probabilities to documents that contain unseen words in the future.

The simplest technique, and the only one you will need for this course, is called **Laplace smoothing** and it's based on the idea that every single outcome (in the data or outside the data) gets an extra virtual count $\alpha \ge 0$ occurrences.

To implement Laplace smooth effectively and more easily, it is convenient to augment the vocabulary of the language with placeholder token (usually denoted UNK for *unknown*), and whenever we encounter an unknown word in the future (e.g., in some test data) we treat it as if it were the UNK word. This effectively increases the vocabular size by 1.


We can update the MLE estimates to use Laplace smoothing as follows:
<details>
    <summary><b>Click to reveal</b></summary>
    
    
\begin{align}
\pi^{(y)}_{w} = \frac{\mathrm{count}_{YX}(y, w) + \alpha}{\alpha \times |\mathcal W| + \sum_{v \in \mathcal W}\mathrm{count}_{YX}(y, v)}
\end{align}

where we assume the vocabulary $\mathcal W$ includes a special UNK symbol.
    
---
    
</details>



## <a name="complexity"> Complexity

    
Let's check if you are getting comfortable with the formal presentation of the model (which is in principle enough to answer the following quizzes).

**Quiz** What is the time complexity of assessing joint probability $P_{YX}(y, w_{1:l})$?

<details>
    <summary><b>SOLUTION</b></summary>
    
It takes $\mathcal O(l)$, since we have to assess $P_Y(y)$ once and $P_{W|Y=y}$ once for every word in $w_{1:l}$.

---
    
</details>



**Quiz** What is the time complexity of assessing marginal probability $P_X(w_{1:l})$?

<details>
    <summary><b>SOLUTION</b></summary>
    
It takes $\mathcal O(K \times l)$, since we need to assess the joint $P_{YX}(y, w_{1:l})$ once for every label in the sample space $\{1, \ldots, K\}$. This is necessary so we can marginalise $Y$ out.
    
---
    
</details>



**Quiz** What is the storage requirements of the NBC model?

<details>
    <summary><b>SOLUTION</b></summary>
    
It requires space dominated by $\mathcal O(K \times V)$ since we have to store a $K$-dimensional vector $\phi_{1:K}$ and $K$ class-conditioned cpds, each defined over $V$ words.
    
---
    
</details>



**Quiz** What is the time complexity to classify $w_{1:l}$ with a trained model?

<details>
    <summary><b>SOLUTION</b></summary>
    
To classify, we need to assess the joint probability $P_{YX}(y, x_{1:l})$ once for each label in the sample space $\{1, \ldots, K\}$, and then pick the one which is maximum. This takes time $\mathcal O(K \times l)$ since each assessment of joint probability is linear in $l$ and finding the maximum in a list is not worse than linear.
    
---
    
</details>

